In [7]:
# Cell 1 - Install / upgrade LangGraph + deps (best-practice pinned install)
# If you want the absolute latest, you can remove the ==1.0.2 pin, but pinning avoids surprises.
!pip install -qU langchain python-dotenv graphviz
!pip install -U langgraph

# Verify versions programmatically
import importlib, pkgutil, sys
import langgraph, langchain
from packaging import version

print("python:", sys.version.splitlines()[0])
print("langchain:", getattr(langchain, "__version__", "unknown"))

python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
langgraph: unknown
langchain: 1.0.1


In [8]:
!pip show langgraph

Name: langgraph
Version: 1.0.2
Summary: Building stateful, multi-actor applications with LLMs
Home-page: 
Author: 
Author-email: 
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langgraph-checkpoint, langgraph-prebuilt, langgraph-sdk, pydantic, xxhash
Required-by: langchain


In [3]:
from google.colab import userdata
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")


In [18]:
# Cell 2 - Try correct imports; helper to re-install if import fails
try:
    # Correct modern entrypoints
    from langgraph.graph import StateGraph, START, END, add_messages
    print("✅ Imported StateGraph and constants successfully.")
except Exception as e:
    print("❌ Import failed:", e)
    print("Attempting to upgrade langgraph to the latest version and re-import...")
    !pip install -qU langgraph
    # retry import
    from importlib import reload
    import langgraph as _lg
    reload(_lg)
    from langgraph.graph import StateGraph, START, END, add_messages
    print("✅ Re-imported after upgrade.")

✅ Imported StateGraph and constants successfully.


In [19]:
# Cell 3 - LLM selector (Gemini or OpenAI). Put keys in environment (Colab: use %env or set via os.environ)
from google.colab import userdata
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

if not (GOOGLE_API_KEY):
    print("⚠️ No LLM API key detected. You can still run the graph in a 'mock' mode below.")
else:
    print("Detected LLM keys:", "Google" if GOOGLE_API_KEY else "")

# LLM wrapper function (keeps notebook LLM-agnostic)
def get_llm():
    if GOOGLE_API_KEY:
        from langchain_google_genai import ChatGoogleGenerativeAI
        return ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=GOOGLE_API_KEY)
    else:
        class MockLLM:
            def invoke(self, prompt):
                return type("R", (), {"content": f"[MOCK OUTPUT: {prompt[:60]}...]"})
        return MockLLM()

llm = get_llm()
print("✅ Using:", type(llm).__name__)

Detected LLM keys: Google
✅ Using: ChatGoogleGenerativeAI


In [20]:
from typing_extensions import TypedDict

class State(TypedDict):
    text: str
    summary: str
    sentiment: str

def call_llm(prompt: str):
    """Unified LLM call safe for both OpenAI and Gemini adapters."""
    if hasattr(llm, "invoke"):
        try:
            return llm.invoke(prompt).content
        except Exception as e:
            # Gemini client errors often arise from internal keyword differences
            print("⚠️ invoke() failed:", e)
    if hasattr(llm, "generate"):
        try:
            return llm.generate([prompt]).generations[0][0].text
        except Exception as e:
            print("⚠️ generate() failed:", e)
    # fallback for mock mode
    return f"[MOCK OUTPUT: {prompt[:60]}...]"

def node_summarize(state: State):
    prompt = f"Summarize this text in one concise sentence:\n\n{state['text']}"
    state["summary"] = call_llm(prompt)
    return state

def node_sentiment(state: State):
    prompt = f"Determine sentiment (Positive/Negative/Neutral):\n\n{state['text']}"
    state["sentiment"] = call_llm(prompt)
    return state

print("✅ Node functions updated for modern LLM wrappers.")

✅ Node functions updated for modern LLM wrappers.


In [21]:
# Cell 5 - Build and compile a StateGraph
builder = StateGraph(State)

# Add nodes and set entry/finish (modern pattern uses set_entry_point / set_finish_point)
builder.add_node("summarize", node_summarize)
builder.add_node("sentiment", node_sentiment)

builder.set_entry_point("summarize")
builder.set_finish_point("sentiment")

# connect summarize -> sentiment
builder.add_edge("summarize", "sentiment")

graph = builder.compile()
print("✅ Graph compiled. Entry:", "summarize", "Finish:", "sentiment")

✅ Graph compiled. Entry: summarize Finish: sentiment


In [22]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(State)
builder.add_node("summarize", node_summarize)
builder.add_node("sentiment", node_sentiment)
builder.set_entry_point("summarize")
builder.set_finish_point("sentiment")
builder.add_edge("summarize", "sentiment")

graph = builder.compile()

result = graph.invoke({
    "text": "LangGraph is a modern framework for building stateful AI workflows.",
    "summary": "",
    "sentiment": ""
})

import json
print(json.dumps(result, indent=2))

{
  "text": "LangGraph is a modern framework for building stateful AI workflows.",
  "summary": "LangGraph is a modern framework for building stateful AI workflows.",
  "sentiment": "Positive"
}


In [23]:
# Cell 7 - Conditional branching example
def praise_node(state: State):
    print("PRAISE:", state["summary"])
    return state

def critique_node(state: State):
    print("CRITIQUE:", state["summary"])
    return state

builder2 = StateGraph(State)
builder2.add_node("summarize", node_summarize)
builder2.add_node("sentiment", node_sentiment)
builder2.add_node("praise", praise_node)
builder2.add_node("critique", critique_node)

builder2.set_entry_point("summarize")
builder2.set_finish_point("praise")  # we will route to praise or critique dynamically

builder2.add_edge("summarize", "sentiment")

# routing function
def choose_route(state: State):
    s = (state.get("sentiment","") or "").lower()
    if "positive" in s:
        return "praise"
    else:
        return "critique"

# add conditional edges using current API
builder2.add_conditional_edges("sentiment", choose_route, {"praise":"praise", "critique":"critique"})
builder2.add_edge("praise", END)
builder2.add_edge("critique", END)

g2 = builder2.compile()
print("Compiled conditional graph.")
g2.invoke({"text":"I love this API! It's great.","summary":"","sentiment":""})

Compiled conditional graph.
PRAISE: The API is highly praised.


{'text': "I love this API! It's great.",
 'summary': 'The API is highly praised.',
 'sentiment': 'Sentiment: **Positive**'}

In [27]:
!pip install -U langgraph

In [29]:
from IPython.display import Image, display
import tempfile, os, subprocess

app = builder.compile()

try:
    # Try to get Mermaid syntax
    dot_or_mermaid = app.get_graph().draw_mermaid()
except Exception as e:
    print("❗ Could not call draw_mermaid():", e)
    try:
        # Fallback to older PNG bytes
        img_bytes = app.get_graph().draw_mermaid_png()
        display(Image(img_bytes))
        dot_or_mermaid = None
    except Exception as e2:
        print("❗ Could not call draw_mermaid_png():", e2)
        dot_or_mermaid = None

if isinstance(dot_or_mermaid, str):
    # we have Mermaid source
    print("Mermaid code (paste into mermaid.live):")
    print(dot_or_mermaid)

Mermaid code (paste into mermaid.live):
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	summarize(summarize)
	sentiment(sentiment)
	__end__([<p>__end__</p>]):::last
	__start__ --> summarize;
	summarize --> sentiment;
	sentiment --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

